# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Import-dependencies-and-data" data-toc-modified-id="Import-dependencies-and-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import dependencies and data</a></div><div class="lev1 toc-item"><a href="#Functions" data-toc-modified-id="Functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Functions</a></div><div class="lev1 toc-item"><a href="#Examples" data-toc-modified-id="Examples-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Examples</a></div><div class="lev2 toc-item"><a href="#Synchronizing-receiver-with-sync-tag" data-toc-modified-id="Synchronizing-receiver-with-sync-tag-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Synchronizing receiver with sync tag</a></div><div class="lev2 toc-item"><a href="#Synchronizing-receiver-without-synctag" data-toc-modified-id="Synchronizing-receiver-without-synctag-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Synchronizing receiver without synctag</a></div>

# Import dependencies and data

In [ ]:
import numpy as np
import pandas as pd 
import pytz
from scipy.interpolate import UnivariateSpline, splrep
import pickle


In [ ]:
with open('VPS_receivers_subset.pkl', 'rb') as handle:
    receiverlogs = pickle.load(handle)

In [ ]:
all_watertemp_data = pd.read_pickle('all_watertemp_data.pkl')
all_salinity_data = pd.read_pickle('all_salinity_data.pkl')

# Functions

In [ ]:
def DTD_Sx(Sx_on_rec, Sx_on_base, tcol, outlier_lim=0.001):
    
    """
    Function to calculate the Detection Time Difference of a transmitter versus a receiver pair.
    
    Parameters
    ----------
    Sx_on_rec : df with synctag detections on first receiver (THIS IS THE RECEIVER YOU WANT TO SYNCHRONISE)
    Sx_on_base : df with synctag detections of same synctag on second receiver (BASE STATION)
    tcol : name of time column
    outlier_lim: outlier is removed if difference with previous point >= remove_outliers
                 default 0.001
                 
    Returns
    -------
    DTD_res : df with time in index and DTD in rows
    """
    
    Sx_on_rec['lower_lim'] = Sx_on_rec[tcol]-pd.Timedelta(seconds=100)
    Sx_on_rec['upper_lim'] = Sx_on_rec[tcol]+pd.Timedelta(seconds=100)
    
    Sx_on_base = Sx_on_base.set_index(tcol)
    
    def get_corres_point(row):
        """
        function to get corresponding time on the second receiver of a detection on the first receiver
        """
        try:
            return Sx_on_base[row.lower_lim:row.upper_lim].index[0]
        except IndexError: # if there is no point in interval
            return None

    Sx_on_rec['base_time'] = Sx_on_rec.apply(get_corres_point, axis=1) 
    
    # calculate Detection Time Difference between corresponding points
    Sx_on_rec['DTD'] = (Sx_on_rec.Time - Sx_on_rec.base_time).apply(lambda x: x.total_seconds())
    
    # resulting dataframe
    DTD_res = Sx_on_rec[['Time','DTD']].set_index('Time')
    DTD_res = DTD_res[DTD_res.diff().abs()<outlier_lim] # remove outliers, needed for modelling the spline
    
    return DTD_res

In [ ]:
def model_spline(DTD_S, ts, k=3, s=0.0003):
    """
    Function to model the spline (error on DTD due to receiver drift)
    
    Parameters
    ----------
    DTD_S : df with time in index and DTD between 2 receivers in rows (result from DTD_Sx)
    ts : timeseries on which to apply the modelled spline
    k : Degree of the smoothing spline.  Must be <= 5.
    Default is k=3, a cubic spline.
    s : Positive smoothing factor used to choose the number of knots. 
    Default value of 0.0003 gives good equilibrum between interpolating through each point
    and smoothing slightly (but also requires outliers >0.001 to be removed)
    
    
    Returns
    -------
    ys : modelled spline
    """

    x = DTD_S[DTD_S.notna().values].index.astype(int)
    y = DTD_S[DTD_S.notna().values]['DTD'].values
    s = UnivariateSpline(x, y, k=k, s=s)
    
    ys = s(np.int64(ts))
    
    return ys

In [ ]:
def calc_soundspeed(T, S, z):
    """
    Equation of Mackenzie: http://asa.scitation.org/doi/10.1121/1.386919
    parameters
    ----------
    T = temperatuur in °C
    S = salinity in parts per thousand
    z = depth in meters
    
    returns
    ------
    soundspeed in m/s
    """

    a1 = 1448.96
    a2 = 4.591
    a3 = -5.304e-2
    a4 = 2.374e-4
    a5 = 1.34
    a6 = 1.63e-2
    a7 = 1.675e-7
    a8 = -1.025e-2
    a9 = -7.139e-13
    
    SS = a1 + a2*T + a3*T**2 + a4*T**3 + a5*(S-35) + a6*z + a7*z**2 + a8*T*(S-35) + a9*T*z**3
    return SS

# Examples

## Synchronizing receiver with sync tag

The base station is ST08. As an example, ST15 is synchronised, based on mutual detection of S8 and S15 (the respective sync tags).  
- ST08: 127025
- ST15: 127033
- S15: A69-1601-65043
- S8: A69-1601-65046

In [ ]:
rec_ST15 = receiverlogs['127033']
rec_ST08 = receiverlogs['127025']

In [ ]:
S15_on_rec_ST15 = rec_ST15[rec_ST15.Transmitter == 'A69-1601-65043'].reset_index(drop=True)
S15_on_rec_ST08 = rec_ST08[rec_ST08.Transmitter == 'A69-1601-65043'].reset_index(drop=True)

DTD_S15 = DTD_Sx(Sx_on_rec=S15_on_rec_ST15, Sx_on_base=S15_on_rec_ST08, tcol='Time')

In [ ]:
S8_on_rec_ST15 = rec_ST15[rec_ST15.Transmitter == 'A69-1601-65046'].reset_index(drop=True)
S8_on_rec_ST08 = rec_ST08[rec_ST08.Transmitter == 'A69-1601-65046'].reset_index(drop=True)

DTD_S8 = DTD_Sx(Sx_on_rec=S8_on_rec_ST15, Sx_on_base=S8_on_rec_ST08, tcol='Time')

In [ ]:
ts = rec_ST15.Time
ys_ST = model_spline(DTD_S=DTD_S15, ts=ts)
ys_base_ST = model_spline(DTD_S=DTD_S8, ts=ts)
spline = (ys_ST+ys_base_ST)/2
rec_ST15['synced_time'] = rec_ST15.Time - pd.to_timedelta(spline, unit='s')

In [ ]:
rec_ST15.head()

## Synchronizing receiver without synctag

The base station is ST08. As an example, ST09 is synchronised, based on mutual detection of S8 and on the distance between ST09 and ST08.   
- ST09: 124064
- S8: A69-1601-65046
- distance = 65.9 m

In [ ]:
rec_ST09 = receiverlogs['124064']

In [ ]:
S = 'S8'
#S8_on_rec_ST08 = rec_ST08[rec_ST08.Transmitter == 'A69-1601-65046'].reset_index(drop=True)
S8_on_rec_ST09 = rec_ST09[rec_ST09.Transmitter == 'A69-1601-65046'].reset_index(drop=True)

DTD_S8b = DTD_Sx(Sx_on_rec=S8_on_rec_ST09, Sx_on_base=S8_on_rec_ST08, tcol='Time')

In [ ]:
ts = rec_ST09.Time
ts = ts.apply(lambda x: pytz.utc.localize(x))

 # match water temperature/salinity datetimes to receiver datetimes
temp_matches = all_watertemp_data.reindex(ts, method='nearest')
sal_matches = all_salinity_data.reindex(ts, method='nearest')
all_matches = pd.concat([temp_matches, sal_matches], axis=1)

def ss_on_row(row):
    return calc_soundspeed(T=row.Temp, S=row.Salinity_ppt, z=1)

soundspeed = list(all_matches.apply(ss_on_row, axis=1)) # takes about 11s for receiver ST9
dist = 65.9
DTD_from_dist = np.divide(dist,soundspeed)

ys_base_ST = model_spline(DTD_S=DTD_S8b, ts=ts)
spline = ys_base_ST - DTD_from_dist
rec_ST09['synced_time'] = rec_ST09.Time - pd.to_timedelta(spline, unit='s')

In [ ]:
rec_ST09.head()